caffe2发布后，将pytorch迁移到caffe2是一个静态图框架，能在手机中运行模型
ONNX：开放式可交换神经网络

虽然所有深度学习框架都建立在自动微分和计算图的基础上，但有两种截然不同的实现方法:静态图和动态图

静态图:在数据实际拿上去进行计算时，先生成计算图实例(Tensoflow使用内部引擎构建优化的计算图)

## 使用静态图 tensorflow.compat.v1

In [ ]:
import tensorflow.compat.v1 as tf
tf.compat.v1.disable_eager_execution()

lr = 0.001
x = tf.placeholder(tf.float32,shape=[None,2],name='x-input')
y = tf.placeholder(tf.float32,shape=[None,2],name='y-input')
w1 = tf.Variable(tf.random_uniform([2,5],-1,-1),name='w1') #XOR运算
w2 = tf.Variable(tf.random_uniform([5,2],-1,-1),name='w2') #XOR运算
b1 = tf.Variable(tf.zeros([5]),name='b1')
b2 = tf.Variable(tf.zeros([2]),name='b2')
a2 = tf.sigmoid(tf.matmul(x,w1)+b1)
hyp = tf.matmul(a2,w2) + b2
cost = tf.reduce_mean(tf.losses.mean_squared_error(y,hyp))
train_step = tf.train.GradientDescentOptimizer(lr).minimize(cost)
prediction = tf.argmax(tf.nn.softmax(hyp),1)
#一旦解释器完成了读取图定义，我们就可以开始循环访问数据了
#接下来启用一个tensorflow对话，这是可以与实现生成的图交互的唯一方式
'''
with tf.Session() as sess:
    sess.run(init) #使用session.run的方法将数据传递到图
    for i in range(epoch):
        sess.run(train_step,feed_dict={x_:XOR_X,y_:XOR_Y})
'''

## 使用动态图 torch
    动态功能是命令式图的构建风格，动态图架构不会在传送数据之前生成图，程序在每次迭代数据后都会生成一个新的图实例，并在反向过程完成后将其销毁。
    但是你无法根据假设对图进行预优化

In [ ]:
#自动编码器是一种特殊的编码器-解码器网络，属于无监督学习范畴，自动编码器尝试从未标记的数据中进行学习。如果能够训练出一个这样的网路，-就会找到一个好的压缩算法
#其可以将高维量变为低维向量存储
#语义分割就用到了编码器

torch的基本数据对象是tensor对象:

| 数据类型 | CPU张量 | GPU张量
| -- | -- | --
| 32位浮点型 | torch.FloatTensor | torch.cuda.FloatTensor
| 64位浮点型 | torch.DoubleTensor | torch.cuda.DoubleTensor
| 8位整型(无符号) | torch.ByteTensor | torch.cuda.ByteTensor
| 8位整型(有符号) | torch.CharTensor | torch.cuda.CharTensor
| 16位整型(有符号) | torch.ShortTensor | torch.cuda.ShortTensor
| 32位整型(有符号) | torch.IntTensor | torch.cuda.IntTensor
| 64位整型(有符号) | torch.LongTensor | torch.cuda.LongTensor


In [ ]:
import torch
a = torch.cuda.FloatTensor([1,1,1]) #size,shape,storage都可以查看数据格式
#PyTorch遵循为同一操作使用后缀下划线的惯例，但这会发生in-place
# a.add(b) 不会对a,b做任何操作
# a.add_(b) 会更新a
b  = torch.cuda.FloatTensor([1,1,1])

x = torch.rand(2,3)
xx_0 = torch.cat((x,x))
xx_1 = torch.cat((x,x),dim=1)
xx_squeeze =

# 一个简单的神经网络

In [ ]:
#PyTorch提供了很好的数据预处理封装器
import torch
inputs = torch.FloatTensor([2])
weights = torch.rand(1,requires_grad=True)
bias = torch.rand(1,requires_grad=True)
t = inputs @ weights
out = t + bias
out.backward()
weights.grad 
'''
.grad存储任意节点在任意时间点的梯度
.data用于访问包含数据的裸张量对象
在创建张量时，可以指定该张量是否需要梯度
输入的required_grad=False
权重和偏差的required_bias=True
'''
'''
张量实例和Function是在图中相互连接的，他们一起构成了非循环计算图
除了用户明确定义的张量以外，每个张量都与一个函数相连

当我们在遍历数据时，实际是在构件图，并在到达最后一个节点时进行反向传播
我们用 lr* .grad 来更新 .data
'''
'''
在每次反向传播之后，我们都要清空梯度
每一个张量都有 .data .grad .grad_fn 三个属性，我们通过grad_fn来链接张量
'''




![title](https://pic2.zhimg.com/80/v2-0a938a33a77b14171cb17f2bbafc0ba1_720w.jpg)

In [ ]:
#pytorch通过torch.nn模块提供了高阶API
import torch.nn as nn
class FizBuzNet(nn.Module):
    """
    2 layer network for predicting fiz or buz
    param: input_size -> int
    param: output_size -> int
    """
    def __init__(self,input_size,hidden_size,output_size):
        super(FizeBuzNet,self).__init__()
        self.hidden = nn.Linear(input_size,output_size)
        self.out = nn.Linear(hidden_size,output_size)
    def forward(self,batch):
        hidden = self.hidden(batch)
        activated = torch.sigmoid(hidden)
        out = self.out(activated)
        return out
# 我们已经定义了FizBuzNet结构，并将其封装在从torch.nn.Module继承的Python类中
# nn.Module实现了两个主要功能 __call__和backward()  用户需要自己写forward和__init__()
# 一旦返回网络层的初始化对象，就可以通过调用model对象本身将输入数据传递给模型

- apply() 此函数可以帮助我们将自定义的函数应用于模型的所有参数，它通常用于自定义权重的初始化.

- cuda()和cpu():
model.cpu()将所有参数转化成CPU张量
net = FizBuzNet(input_size,hidden_size,output_size)
net.cuda()

- train()和eval():
这些函数告诉pytorch，模型正在训练模式或者评估模式下运行

- parameters():
调用parameters会返回模型的所有参数


In [ ]:
#有一种十分重要的方法叫序列容器，它提供了一种更简单的API来构建模型对象，如果模型是连续且直接的，则无需用户编写代码来实现类结构
import torch.nn as nn

i = 10
h = 5
o = 1
net = nn.Sequential(
    nn.Linear(i,h),
    nn.Sigmoid(),
    nn.Linear(h,o),
    nn.Sigmoid()
)

# nn.functional 模块
nn.functional模块具有将网络节点连接在一起的操作
functional模块具有更多功能

functional模块也有网络层，但是他比nn提供的网络层抽象性要低，而比我们构建新手模型的方式的抽象性要高

In [ ]:
import torch
import torch.nn.functional as F
a = torch.Tensor([[1,1]])
w1 = torch.Tensor([[2,2]])
F.linear(a,w1) == a.matmul(w1.t())

# 深度学习工作流

- 工作流中的部署测试是人们最挣扎的地方，尤其是应用程序的规模相当大时，如前所述，虽然PyTorch是作为一个面向研究框架构建的，但社群成功的将caffe2集成到了Pytorch后端
- 在第八章中，我们通过使用ONNX，PyTorch JIT等的示例来全面讨论模型交付生产的相关内容

- DataLoader接收从torch.utils.data.Dataset继承的dataset类，并进行复杂的操作，比如：小批次处理、多线程、随机打乱
  num_workers决定该操作多少个并行线程来获取数据，这决定了CPU能否跟得上GPU的速度
## 我们需要的三种数据：预训练模型、预处理过的数据集、处理数据的实用函数

In [ ]:
#构建数据处理流程也是一项有难度的任务，在进行一步处理之前，需要将数据转换为浮点数或整数，并进行归一化等等
'''
PyTorch通过提供抽象类来编写自定义数据集和数据加载器
Pytorch中的dataset类是一个高级抽象，它可以处理数据加载其所需要的几乎所有内容
用户自定义的dataset类需要覆盖父类的__len__函数和__getitem__函数
使用__len__获取数据长度 使用__getitem__来获取数据(输入是索引，输出是数据)
'''
from dataclasses import dataclass #通过使用动态代码帮助消除__init__中的样板代码
from torch.utils.data import Dataset,DataLoader

@dataclass(eq=False)
class FizBuzDataset(Dataset):
    input_size: int 
    start: int = 0
    end: int = 1000
    def encoder(self,num):
        ret = [int(i) for i in '{0:b}'.format(num)]
        return[0] * (self.input_size-len(ret)) + ret
    
    def __getitem__(self,idx):
        x = self.encoder(idx)
        if idx % 15 == 0:
            y = [1,0,0,0]
        elif idx % 5 == 0:
            y = [0,1,0,0]
        elif idx % 3 == 0:
            y = [0,0,0,1]
        return x,y
    def __len__(self):
        return self.end - self.start
#数据由程序生成，数据生成的实现位于__getitem__函数中

dataset = FizBuzDataset()
for i in range(len(dataset)):
    x,y = dataset[i]

dataloader = Dataloader(dataset,batch_size=10,shuffle=True,num_workers=True)
for batch in dataloader:
    print(batch)
        

### torchvision、torchtext、torchaudio
#### torchvision具有强大的API，由(1)数据集，(2)预训练模型，(3)预构建的转换脚本 组成
常用的数据集：

| 数据集 | 描述
| -- | --
| MNIST | 70000个 28 * 28 * 1的手写数字
| KMNIST | 排列形式类似MNIST的平假名字符数据集
| COCO | 大规模的用于目标检测、分割和字幕数据集
| LSUN | 大规模场景理解数据集，和COCO类似
| Imagenet12 | 2012年图像识别挑战 1400万张图片
| CIFAR | 60000张32 * 32 * 3的图片

In [ ]:
import torchvision as tv
import torch
import matplotlib.pyplot as plt


#mnist = tv.datasets.MNIST('.',download=True)
#plt.imshow(mnist[0][0])

#torchvision使用Pillow作为加载图像的默认后端，torchvison提供的所有数据都继承了torch.utils.data.Dataset类,因此每个数据都实现了__len__和__getitem__
#这两个神奇的函使所有数据都与Dataloader兼容


#  将本地的图像数据封装成 Dataset 进一步传给 DataLoader

In [ ]:
import torchvision
import torch
import matplotlib.pyplot as plt

#images = tv.datasets.ImageFolder('../input/chinese-mnist/data')
'''
root/dog/xxx.png
root/dog/xxy.png
root/dog/xxz.png
root/cat/123.png
root/cat/nsdf3.png
root/cat/asd932_.png
'''
plt.imshow(images[0][0])

# 使用预训练模型

In [ ]:
resnet18 = torchvision.models.resnet18(pretrained=True)
#把参数冻结
for param in resnet18.layer1.parameters():
    param.requires_grad = False

# 预处理模块

In [ ]:
import torchvision.transforms as transforms

mean = 0
std = 1
transform = transforms.Compose(
[
    transforms.ToTensor(),
    transforms.Normalize(mean,std)
]
)

# 模型实现
- 在本节中，我们将讨论pytorch包本身提供的分析和瓶颈工具以及pytorch推荐的训练实用程序ignite
- 当模型开始表现不佳时，ignite(训练模块)是一个很好的帮助工具

In [ ]:
'''
with torch.autograd.profiler.profile() as prof:
    hyp = resnet18(x)
print(prof)
print(prof.key_averages())
print(prof.tabel('cpu_time'))
prof.export_chrome_trace('chrometrace')
'''

#### pytorch社群创建了两个工具：torchnet 和 ignite，这里只关注ignite

ignite是一种神经网络训练工具，ignite的核心是Engine模块

- (1) 它基于默认/自定义训练器或评估器运行模型
- (2) 它可以处理程序和评价指标，并对其进行操作
- (3) 它可以触发和执行回调

#### ignite开辟了一种特殊的方式，通过事件或者触发器与循环进行交互
它通过修饰器来触发，以下事件是触发事件:
- EPOCH_STARTED
- EPOCH_COMPLETED
- STARTED
- COMPLATED
- ITERATION_STARTED
- ITERATION_COMPLETED
- EXCEPTION_RAISED

# CNN
构建完整的CNN需要四种类型的操作:
- 卷积层
- 非线性层
- 池化层
- 全连接层
![title](https://gimg2.baidu.com/image_search/src=http%3A%2F%2Fjeit.ie.ac.cn%2FfileDZYXXXB_ONLY%2Fjournal%2Farticle%2Fdzyxxxb%2Fnewcreate%2F201029-3_mini.jpg&refer=http%3A%2F%2Fjeit.ie.ac.cn&app=2002&size=f9999,10000&q=a80&n=0&g=0n&fmt=jpeg?sec=1642772571&t=321f41a71dd7947b92c43d17e22ad4d7)

In [ ]:
#   下载数据 + 转换数据 封装到get_data函数下
import torchvision
import torch
import torchvision.transforms as transforms
import matplotlib.pyplot as plt


def get_data():
    transform = transforms.Compose(
        [transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5),(0.5,0.5,0.5))]
    )
    
    trainset = torchvision.datasets.CIFAR10(root='.',train=True,download=True,transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset,batch_size=1,shuffle=True,num_workers=2)
    
    testset = torchvision.datasets.CIFAR10(root='.',train=False,download=True,transform=transform)
    testloader = torch.utils.data.DataLoader(testset,batch_size=1,shuffle=False,num_workers=2)
    
    return trainloader,testloader
   
get_data()   

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# 模型定义
class simple_cnn(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,6,5)
        self.pool = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*5*5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1,16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = simple_cnn()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(),lr=0.001,momentum=0.9)
train_loader,test_loader = get_data()


MAX_EPOCH = 10
train_curve = list()
valid_curve = list()
log_interval = 50

for epoch in range(MAX_EPOCH): 
    loss_mean = 0.
    correct = 0.
    total = 0.
    net.train()
    for i, data in enumerate(train_loader):
        # forward
        inputs, labels = data
        outputs = net(inputs)
        # backward
        optimizer.zero_grad()
        loss = loss_fn(outputs, labels)
        loss.backward()
        # update weights
        optimizer.step()
        # 统计分类情况
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).squeeze().sum().numpy()
        # 打印训练信息
        loss_mean += loss.item()
        train_curve.append(loss.item())
        if (i+1) % log_interval == 0:
            loss_mean = loss_mean / log_interval
            print("Training:Epoch[{:0>3}/{:0>3}] Iteration[{:0>3}/{:0>3}] Loss: {:.4f} Acc:{:.2%}".format(
                epoch, MAX_EPOCH, i+1, len(train_loader), loss_mean, correct / total))
            loss_mean = 0.
 


# 语义分割

- 我们将CamVid数据集用于LinkNet的实现
- Linknet是由编码器+解码器组成的，LinkNet论文的作者将resnet18作为编码器

### 我们主要使用了五个组件来构建linknet
- ConvBlock 卷积 + 非线性
- DeconvBlock 反卷积 + 非线性
- nn.MaxPool2d 池化
- EncoderBlock
- DecoderBlock

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class ConvBlock(nn.Module):
    def __init__(self,inp,out,kernal,stride,pad,bias,act):
        super().__init()
        if act:
            self.conv_block = nn.Sequential(
                nn.Conv2d(inp,out,kernal,stride,pad,bias=bias),
                nn.BatchNorm2d(num_features=out),
                nn.ReLU()
            )
        else:
            self.conv_block = nn.Sequential(
                nn.Conv2d(inp,out,kernal,stride,pad,bias=bias),
                nn.BatchNorm2d(num_features=out),
            )
    def forward(self,x):
        return seld.conv_block(x)
    

class DeconvBlock(nn.Module):
    def __init__(self,inp,out,kernal,stride,pad):
        super().__init__()
        self.conv_transpose = nn.nn.ConvTranspose2d(inp,out,kernal,stride,pad)
        self.batchnorm = nn.BatchNorm2d(out)
        self.relu = nn.ReLU()
    def forward(self,x,output_size):#outputsize是期望输出的维度
        convt_out = self.conv_transpose(x,output_size=output_size)
        batchnormout = self.batchnorm(convt_out)
        return self.relu(batchnormout)

class EncoderBlock(nn.Module):
    'Resnet18'
    def __init__(self,inp,out):
        super().__init__()
        self.block1 = nn.Sequential(
            ConvBlock(inp=inp,out=out,kernal=3,stride=2,pad=1,bias=True,act=True),
            ConvBlock(inp=out,out=out,kernal=3,stride=1,pad=1,bias=True,act=True)
        )
        
        self.block2 = nn.Sequential(
            ConvBlock(inp=out,out=out,kernal=3,stride=1,pad=1,bias=True,act=True),
            ConvBlock(inp=out,out=out,kernal=3,stride=2,pad=1,bias=True,act=True)
        )
        
        self.residue = ConvBlock(inp=out,out=out,kernal=3,stride=2,pad=1,bias=True,act=True)
    
    def forward(self,x):
        out1 = self.block1(x)
        residue = self.residue(x)
        out2 = self.block2(out1+residue)
        return out1 + out2

class DecoderBlock(nn.Module):
    def __init__(self,inp,out):
        super().__init__()
        self.conv1 = ConvBlock(inp=inp,out=inp//4,kernel=1,stride=1,pad=0,bias=True,act=True)
        self.deconv = DeconvBlock(inp=inp//4,out=inp//4,kernel=3,stride=2,pad=1)
        self.conv2 = ConvBlock(inp=inp,out=out,kernel=1,stride=1,pad=0,bias=True,act=True)
    def forward(self,x,output_size):
        conv1 = self.conv1(x)
        deconv = self.deconv(conv1,output_size=output_size)
        conv2 = self.conv2(deconv)
        return conv2

class SegmentationModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.init_conv = Convblock(inp=3,out=64,kernel=7,stride=2,pad=3,bias=True,act=True)
        self.init_maxpool = nn.MaxPool2d(kernel_szie=3,stride=2,padding=1)
        
        self.encoder1 = EncoderBlock(inp=64,out=64)
        self.encoder2 = EncoderBlock(inp=64,out=128)
        self.encoder3 = EncoderBlock(inp=128,out=256)
        self.encoder4 = EncoderBlock(inp=256,out=512)
        
        self.decoder4 = DecoderBlock(inp=512,out=256)
        self.decoder3 = DecoderBlock(inp=256,out=128)
        self.decoder2 = DecoderBlock(inp=128,out=64)
        self.decoder1 = DecoderBlock(inp=64,out=64)
        
        self.final_deconv1 = DeconvBlock(inp=64,out=64)
        self.final_conv = ConvBlock(inp=32,out=32,kernal=3,stride=1,pad=1,bias=True,act=True)
        self.final_deconv2 = DeconvBlock(inp=32,out=2,kernal=2,stride=2,pad=0)
    def forward(self,x):
        init_conv = self.init_conv(x)
        e1 = self.encoder1(init_maxpool)
        e2 = self.encoder2(e1)
        e3 = self.encoder2(e2)
        e4 = self.encoder2(e3)
        
        d4 = self.decoder4(e4,e3.size()) + e3
        d3 = self.decoder4(e4,e3.size()) + e2
        d2 = self.decoder4(e4,e3.size()) + e1
        d1 = self.decoder1(d2,init_maxpool.size())
        
        final_deconv1 = self.final_deconv1(d1,init_conv.size())
        final_conv = self.final_conv(final_deconv1)
        final_deconv2 = self.final_deconv2(final_conv,x.size())
        
        return final_deconv2

# 使用预训练模型进行语义分割
!wget nv url -O path

In [ ]:
!wget -nv http://news.neuq.edu.cn/__local/A/9E/25/208374FD430A5995C27861117FB_D510CA86_3B5BC.jpg -O bird.png
img = Image.open('./bird.png')
img = img.crop((80,0,450,250))
img.save('./bird.png')

In [ ]:
!pip install matplotlib==2.2.3

In [ ]:
#!wget -nv https://img-blog.csdnimg.cn/20200212171003937.jpeg?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3UwMTM2NzkxNTk=,size_16,color_FFFFFF,t_70 -O bird.png
from torchvision import models
#fcn = models.segmentation.fcn_resnet101(pretrained=True).eval()
from PIL import Image
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as T
import numpy as np

def decode_segmap(image, nc=21):
   
  label_colors = np.array([(0, 0, 0),  # 0=background
               # 1=aeroplane, 2=bicycle, 3=bird, 4=boat, 5=bottle
               (128, 0, 0), (0, 128, 0), (128, 128, 0), (0, 0, 128), (128, 0, 128),
               # 6=bus, 7=car, 8=cat, 9=chair, 10=cow
               (0, 128, 128), (128, 128, 128), (64, 0, 0), (192, 0, 0), (64, 128, 0),
               # 11=dining table, 12=dog, 13=horse, 14=motorbike, 15=person
               (192, 128, 0), (64, 0, 128), (192, 0, 128), (64, 128, 128), (192, 128, 128),
               # 16=potted plant, 17=sheep, 18=sofa, 19=train, 20=tv/monitor
               (0, 64, 0), (128, 64, 0), (0, 192, 0), (128, 192, 0), (0, 64, 128)])
 
  r = np.zeros_like(image).astype(np.uint8)
  g = np.zeros_like(image).astype(np.uint8)
  b = np.zeros_like(image).astype(np.uint8)
   
  for l in range(0, nc):
    idx = image == l
    r[idx] = label_colors[l, 0]
    g[idx] = label_colors[l, 1]
    b[idx] = label_colors[l, 2] 
  rgb = np.stack([r, g, b], axis=2)
  return rgb

def segment(net, path):
  img = Image.open(path)
  plt.imshow(img); plt.axis('off'); plt.show()
  # Comment the Resize and CenterCrop for better inference results
  trf = T.Compose([T.Resize(256), 
                   T.CenterCrop(224), 
                   T.ToTensor(), 
                   T.Normalize(mean = [0.485, 0.456, 0.406], 
                               std = [0.229, 0.224, 0.225])])
  inp = trf(img).unsqueeze(0)  
  out = net(inp)['out']
  om = torch.argmax(out.squeeze(), dim=0).detach().cpu().numpy()
  rgb = decode_segmap(om)
  plt.imshow(rgb); plt.axis('off'); plt.show()

segment(fcn,'./bird.png')

# 序列数据处理
- 人类的语言异常复杂，所有词汇的组合超过宇宙中原子的数量，但是深度神经网络通过诸如词嵌入和注意力之类的技术可以很好的处理此问题
- LSTM和GRU(门控循环单元)是最常见的两种RNN
- 下一个重要的发明是Attention,它可以帮助网络专注于输入的重要部分，而不是通过搜索整个输入来找到答案
- 词嵌入是一种革命性思想，它通过比较单词在单词簇中的分布来挖掘单词的概念意义

循环神经网络痛过引入状态变量存储过去的信息和当前的输入，从而可以确定当前的输出

### 自回归模型 : 输入是定的 x(t-T) ~ x(t)
### 隐自回归模型 : 对于过去观测的总结(隐状态) + 当前输入 

In [ ]:
#使用SNLI（stanford natural lauguage inference）作为数据集
#%matplotlib inline
#!pip install d2l
import torch
from torch import nn
import matplotlib.pyplot as plt
from d2l import torch as d2l

T = 1000
time = torch.arange(1,T+1,dtype=torch.float32)
x = torch.sin(0.01*time) + torch.normal(0,0.2,(T,))
#plt.plot(time,x)
#plt.grid()


#将这个序列转换为 特征-标签 对，[x(t-T),...,x(t-1)] 输出y(t)
tau = 4
features = torch.zeros((T-tau,tau))
for i in range(tau):
    features[:,i] = x[i:T-tau+i]
labels = x[tau:].reshape((-1,1))
batch_size,n_train = 16,600
train_iter = d2l.load_array((features[:n_train], labels[:n_train]),batch_size, is_train=True)

#使用·一个·非常简单的架构训练模型：一个拥有两个全连接层的多层感知机，ReLU激活函数和平方损失
'''
nn.init.xavier_uniform_(m.weight)
net = nn.sequential(...)
net.apply(init_weights)
'''
def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
        
def get_net():
    net = nn.Sequential(nn.Linear(4,10),
                        nn.ReLU(),
                        nn.Linear(10,1))
    net.apply(init_weights)
    return net
loss = nn.MSELoss(reduction='none')

def train(net,train_iter,loss,epochs,lr):
    trainer = torch.optim.Adam(net.parameters(),lr)
    for epoch in range(epochs):
        for X,y in train_iter:
            trainer.zero_grad()
            l = loss(net(X),y)
            l.sum().backward()
            trainer.step()
        print('epoch:'+str(epoch + 1) + '  |  '+'loss:'+str(d2l.evaluate_loss(net, train_iter, loss)))
net = get_net()
train(net,train_iter,loss,5,0.01)


#预测
onestep_preds = net(features)
d2l.plot([time, time[tau:]],
         [x.detach().numpy(), onestep_preds.detach().numpy()], 'time',
         'x', legend=['data', '1-step preds'], xlim=[1, 1000],
         figsize=(6, 3))


## 文本预处理
- 1,将文本作为字符串加载到内存中
- 2,将字符串拆分成词元
- 3，建立一个词表，将拆分的词映射到数字索引
- 4，将文本转换为数字索引序列，方便微型操作

In [ ]:
#!pip install d2l
#!pip install matplotlib==2.2.3
import collections
import re
from d2l import torch as d2l
'''
我们从H.G.Well的时光机器中加载文本，只有30000多个单词，足够我们小试牛刀
'''
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt','090b5e7e70c295757f55df93cb0a180b9691891a')

def read_time_machine():
    with open(d2l.download('time_machine'),'r') as f:
        lines = f.readlines()
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]

lines = read_time_machine()
print(f'# 文本总行数: {len(lines)}')
print(lines[0])
print(lines[10])


In [ ]:
'''
tokenize函数将文本行列表(lines)作为输入，列表中的每个元素都是一个文本序列。每个文本序列又被拆分成一个词元列表
词元(token)是文本的基本单位
'''
def tokenize(lines,token='word'):
    if token=='word':
        return [line.split() for line in lines]
    elif token == 'char':
        return [list[line] for line in lines]
    else:
        print('未知词元类型:' + token)

tokens = tokenize(lines)
for i in range(11):
    print(tokens[i])
        

In [ ]:
class Vocab:  #@save
    """文本词表"""
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        # 按出现频率排序
        counter = count_corpus(tokens)
        self._token_freqs = sorted(counter.items(), key=lambda x: x[1],
                                   reverse=True)
        # 未知词元的索引为0
        self.idx_to_token = ['<unk>'] + reserved_tokens
        self.token_to_idx = {token: idx
                             for idx, token in enumerate(self.idx_to_token)}
        self.idx_to_token, self.token_to_idx = [], dict()
        for token, freq in self._token_freqs:
            if freq < min_freq:
                break
            if token not in self.token_to_idx:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1

    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

    @property
    def unk(self):  # 未知词元的索引为0
        return 0

    @property
    def token_freqs(self):
        return self._token_freqs

def count_corpus(tokens):  #@save
    """统计词元的频率"""
    # 这里的tokens是1D列表或2D列表
    if len(tokens) == 0 or isinstance(tokens[0], list):
        # 将词元列表展平成一个列表
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)

#### 在静态图计算中，序列长度必须是给定的，这就是基于静态图的框架与基于NLP任务不兼容的原因，但是Tensorflow提供了名为dynamic_rnn的API来处理此类问题

# 生成网络
生成网络已经在不同的深度学习领域，尤其在计算机视觉领域展现出喜人的成果，一些或活跃的领域包括:去模糊，提高图像分辨率，为图像填充缺失片段，音频去噪，文本生成语音，自动回复消息，文本生成图像
### 1.自回归模型
### 2.GAN
![title](https://gimg2.baidu.com/image_search/src=http%3A%2F%2Foscimg.oschina.net%2Foscnet%2Fc1a9d388-5b87-48db-9dd7-a4cd39f6f354.png&refer=http%3A%2F%2Foscimg.oschina.net&app=2002&size=f9999,10000&q=a80&n=0&g=0n&fmt=jpeg?sec=1642853488&t=45af02ccca6a1cc21326fd9e5182d648)

In [ ]:
import torchvision.transforms as transforms
import torchvision
import torch

def get_data():
    transform = transforms.Compose(
        [transforms.ToTensor()]
    )
    
    trainset = torchvision.datasets.MNIST(root='.',train=True,download=True,transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset,batch_size=1,shuffle=True,num_workers=0)
    
    testset = torchvision.datasets.MNIST(root='.',train=False,download=True,transform=transform)
    testloader = torch.utils.data.DataLoader(testset,batch_size=1,shuffle=False,num_workers=0)
    
    return trainloader,testloader

train_loader,test_loader = get_data()

In [ ]:
#这里只写GAN了嗷
import torch.nn as nn
import torch
import torchvision
import torch.optim as optim


class DiscriminatorNet(nn.Module):
    """
    A three hidden layer Discriminator
    输入是图片784，输出是真假（0，1）
    """
    def __init__(self):
        super().__init__()
        n_features = 784#28*28
        n_out = 1
        
        self.hidden = nn.Sequential(
                nn.Linear(n_features,1024),
                nn.LeakyReLU(0.2),
                nn.Dropout(0.3),
                nn.Linear(1024,512),
                nn.LeakyReLU(0.2),
                nn.Dropout(0.3),
                nn.Linear(512,256),          
                nn.LeakyReLU(0.2),
                nn.Dropout(0.3),
                nn.Linear(256,n_out),
                nn.Sigmoid()
        )
    def forward(self,x):
        x = self.hidden(x)
        return x


def train_discriminator(optimizer,real_data,fake_data):
    optimizer.zero_grad()
    #1,Train on Real Data
    prediction_real = discriminator(real_data)
    error_real = loss(prediction_real,real_data_target(real_data.size(0)))#zhegesize就是
    error_real.backward()
    
    #2,Train on Fake Data
    prediction_fake = discriminator(fake_data)
    error_fake = loss(prediction_fake,fake_data_target(real_data.size(0)))
    error_fake.backward()
    
    #3,update weight with gradients
    optimizer.step()
    #return error
    return error_real + error_fake,prediction_real,prediction_fake

        

def real_data_target(size):
    data = torch.ones(size,1)
    if torch.cuda.is_available():return data.cuda()
    return data
def fake_data_target(size):
    data = torch.zeros(size,1)
    if torch.cuda.is_available():return data.cuda()
    return data
def noise(size):
    n = torch.randn(size,100)
    if torch.cuda.is_available():return n.cuda()
    return n

class GeneratorNet(nn.Module):
    """
    A three hidden layer Generator
    输入是100，输出是784
    """
    def __init__(self):
        super().__init__()
        n_features = 100#28*28
        n_out = 784
        
        self.hidden = nn.Sequential(
                nn.Linear(n_features,256),
                nn.LeakyReLU(0.2),
                nn.Linear(256,512),
                nn.LeakyReLU(0.2),
                nn.Linear(512,1024),          
                nn.LeakyReLU(0.2),
                nn.Linear(1024,n_out),
                nn.Tanh()
        )
    def forward(self,x):
        x = self.hidden(x)
        return x

def train_generator(optimizer,real_data,fake_data):
    
    optimizer.zero_grad()
    picture_fake = generator(fake_data)
    picture_real = real_data
    
    error = loss_g(picture_fake,picture_real)
    error.backward()

    #3,update weight with gradients
    optimizer.step()
    #return error
    return error,picture_real,picture_fake
    
    



In [ ]:
#主函数
discriminator = DiscriminatorNet()
generator = GeneratorNet()
d_optimizer = optim.Adam(discriminator.parameters(),lr=0.0002)
g_optimizer = optim.Adam(generator.parameters(),lr=0.0002)
loss = nn.BCELoss()
loss_g = nn.MSELoss()

discriminator.train()
generator.train()

EPOCH_NUM = 500

for epoch in range(EPOCH_NUM):
    for i, data in enumerate(train_loader):
        
        real_data, _ = data
        real_data = real_data[0][0]
        real_data = real_data.view(1,784)

        fake_data1 = noise(1)
        fake_data2 = generator(fake_data1)
        
        e_d,_,_ = train_discriminator(d_optimizer,real_data,fake_data2)
        e_g,_,_ = train_generator(g_optimizer,real_data,fake_data1)
    print('loss of D:'+str(e_d)+'  loss of G:'+str(e_g))



In [ ]:
x = noise(1)
img = generator(x).detach().numpy()
img.resize(28,28)
plt.imshow(img)


## Cycle Gan
![title](https://gimg2.baidu.com/image_search/src=http%3A%2F%2Fmedia.paperweekly.site%2FLUOHAO_1513259995.6203003.png%3FimageView2%2F2%2Fw%2F800%2Fq%2F70%7Cimageslim&refer=http%3A%2F%2Fmedia.paperweekly.site&app=2002&size=f9999,10000&q=a80&n=0&g=0n&fmt=jpeg?sec=1643168659&t=aaf695141fffe0abe04a634abedf0836)

# 强化学习

- 在整个强化学习范式中，我们只关心如何从交互中学习，决策者被认为是一个智能体
- 智能体必须观察周围的环境并根据环境做出决策或采取行动
- 在环境中实施某个决策得到的响应被称为奖励
- 智能体的目标是在其生命周期内最大化其累积的回报
- 我们必须平衡两个因素，探索和开发 探索是随机性因子，开发是最优决策
- 我们将使用OpenAI的Gym库
- 有明确的结束点的任务成为回合制任务（目标就是回合结束时累计奖励最大）
- 有些任务可能会永远进行下去，这种任务被称为持续任务

### 区分奖励和回报（未来奖励的总和称为回报）
显然，近期回报可能相对更重要一些，我们引入了折扣回报的概念： Goal = Max(R(t+1),gamma*R(t+2),gamma^2*R(t+2)......)
一个好的做法是让 gamma=0.9，他会一直固定，直到试验结束

### 马尔可夫决策过程(MDP)
五个因素： 有限状态，有限行动，有限奖励，折扣率，环境的一步动态
- 花些时间模拟所有行动和状态，来更好地了解他们，通过详细罗列智能体可以处于的所有状态以及在所有状态下可以执行的所有行动和每个行动的概率，我们可以模拟环境，明确所有环境后，环境的一步动态就定了
- 函数V是用来评估某个状态的价值的
- 函数Q是用来预测每个行动的预期回报的
- 假设V和Q任一个都是可以训练的神经网络
-贝尔曼方程指出，在每个时间点的回报等于下一个时间步的估计奖励加上之后状态的折扣奖励

# 将pytorch应用到生产
- onnx(开放神经网络交换)的缺点：onnx的脚本模式，onnx运行一次图来获取有关图的信息，然后将其转为onnx格式，因此，onnx无法迁移模型中的控制流
- 将pytorch用于生产的第二种方法是在pytorch中构建高性能后端
- torchscript将python模型转化为序列化格式，该格式可以加载到高性能空间，torchscript可以被pytorch的后端libtorch读取，

### 本章介绍
- (1) 使用Flask为标准的Pytorch模型提供服务
- (2) 将Pytorch转换为Mxnet，并使用Mxnet提供服务


In [ ]:
# pth转onnx 把之前的语义分割模型转成了onnx格式
inp = [[[[0]*224]*224]*3]
inp[0][0][0][0] = 1
dummy_input = torch.Tensor(inp)
torch.onnx.export(fcn,dummy_input,'./segment.onnx',verbose=True)


In [ ]:
#当你觉得模型服务器(诸如clipper,tensorrt)不好用时，可以试试MXNet的模型服务器

ONNX 是使用跟踪来创建优化的中间表示，也就是说，他通过模型传递一个虚拟输入，当模型被执行时
他记录pytorch的操作，并将这操作转化为中间表示。

与ONNX一样，torchscript可以作为中间结果保存在磁盘中

## 但是，保存的Torchscript模型可以在没有python依赖项的环境中部署加载
- torchscript允许两种方式来生成这种中间表示：
- (1)和ONNX一样，跟踪+记录
- (2)torch.jit.script

# RedisAI